### Run all cells, the interactive plot is at the bottom

In [ ]:
using Interact, StatsPlots, DataFrames, CSV, Dates;
include("plotting_functions.jl");
#WebIO.install_jupyter_nbextension() # might need to restart Jupyter afterwards in order to work

┌ Info: Precompiling Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1260


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15045722742481327864\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-1187022903740518276\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15045722742481327864\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1260
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1260


In [ ]:
Input_df = CSV.read("data/200124_datafile_all_details_right_timestamp.csv");

#### Choose the parameter combination to evaluate in the interactive plot (see results folder)

In [ ]:
h_predict=8760;
h_control=8760;
h_start=1;
h_end=8760;
objective=1;
case=1;
costfactor=1.0;

In [ ]:
Flow_df = CSV.read("results/results_$(h_predict)_$(h_control)_$(h_start)"*
    "-$(h_end)_$(objective)_$(case)_$(costfactor).csv");

In [ ]:
first(Flow_df, 3)

In [ ]:
first(Input_df, 3)

In [ ]:
# merge on data stamp
Data_df = leftjoin(Flow_df, Input_df, on=[:month, :day, :hour], makeunique=false, 
    indicator=nothing, validate=(false, false));
first(Data_df,3)

In [ ]:
size(Data_df)

In [ ]:
@manipulate for start_date=Date(2015,1,1):Day(1):Date(2015,12,31), 
    range_shown=1:1:8
    
    p1=bar_row(bar_PV, Data_df, start_date, range_shown);
    p2=bar_row(bar_demand, Data_df, start_date, range_shown);
    p3=bar_row(bar_heat, Data_df, start_date, range_shown);
    p4=bar_row(bar_comfort, Data_df, start_date, range_shown);
    
    vbox(vskip(1em),
        md"*Interactive illustration of SHEMS results:*",
        vskip(1em),
        plot(p1,p2,p3,p4, layout=(4,1), size=(range_shown*300,4*200)))
end